# Entity Matching Between Product Transaction Name and Product Name Catalog

**Problem Statement**

We have product transaction dataset and product catalog dataset. The existing condition, Product transaction name was input manualy by user. Then the administration team manually matching between product transaction and product catalog name. We want to automate match product transaction name between product catalog name or create new product catalog using python

# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
catalog = pd.read_excel('Product Catalog.xlsx')

In [4]:
pos = pd.read_excel('Product Name from PoS Transactions.xlsx')

In [5]:
catalog['cleaned_name'] = catalog['Product SKU'].str.lower().str.replace('-', ' ').str.replace(',', ' ').str.replace(r'@.*', '', regex=True)
pos['cleaned_name'] = pos['Product Name'].str.lower().str.replace('-', ' ').str.replace(',', ' ').str.replace(r'@.*', '', regex=True)

In [6]:
pos['cleaned_name'] = pos['cleaned_name'].astype(str)
catalog['cleaned_name'] = catalog['cleaned_name'].astype(str)

In [26]:
pos

,Product Name,cleaned_name
0,Pupuk Urea N 46%,pupuk urea n 46%
1,Pupuk Amonium Sulfat ZA,pupuk amonium sulfat za
2,Pupuk Super Fosfat SP-36,pupuk super fosfat sp 36
3,Pupuk NPK Phonska,pupuk npk phonska
4,Pupuk NPK Formula Khusus,pupuk npk formula khusus
...,...,...
43997,Extra one 680 EC @ 250 ml,extra one 680 ec
43998,Extra One 680 SC @ 500 ml,extra one 680 sc
43999,JARING ARWANA @ 100 METER,jaring arwana
44000,Terong Puma F1 @ 5 gram,terong puma f1


In [27]:
catalog

,Product SKU,Brand,Type,Formula,cleaned_name
0,Urea Petro,PIHC,Urea,NaN,urea petro
1,Urea PIM,PIHC,Urea,NaN,urea pim
2,Urea Nitrea,PIHC,Urea,NaN,urea nitrea
3,Urea Daun Buah,PIHC,Urea,NaN,urea daun buah
4,Urea Pusri,PIHC,Urea,NaN,urea pusri
...,...,...,...,...,...
182,Mestical,LaoYing,Nitrogen,NaN,mestical
183,Mesti-GAS,LaoYing,ZA,NaN,mesti gas
184,Mestac,LaoYing,Nitrogen,NaN,mestac
185,MestiKisrit,LaoYing,Mg,NaN,mestikisrit


# Simmiliarity

## Jaccard Calculation

In [7]:
def calculate_jaccard_similarity(a, b):
    a_set = set(a.split())
    b_set = set(b.split())
    return len(a_set.intersection(b_set)) / len(a_set.union(b_set))

In [8]:
# Create an empty list to store the results
result_data = []

for pos_name in pos['cleaned_name']:
    best_similarity = 0
    best_catalog_name = ''

    for catalog_name in catalog['cleaned_name']:
        similarity = calculate_jaccard_similarity(pos_name, catalog_name)
        if similarity > best_similarity:
            best_similarity = similarity
            best_catalog_name = catalog_name

    result_data.append([pos_name, best_catalog_name, best_similarity])

# Create the result DataFrame from the result_data list
result_df = pd.DataFrame(result_data, columns=['pos_cleaned_name', 'best_catalog_name', 'jaccard_similarity_score'])

**Stage I**

filter upper 0.5 score

In [9]:
result_jaccard = result_df[result_df['jaccard_similarity_score']>=0.5]

In [10]:
result_jaccard

,pos_cleaned_name,best_catalog_name,jaccard_similarity_score
11,nitrea,urea nitrea,0.500000
82,mutiara,mutiara 16 16 16,0.500000
84,nitroku,nitroku 16 16 16,0.500000
106,urea,urea petro,0.500000
108,phonska plus,phonska plus 15 15 15+9s+0.2zn,0.500000
...,...,...,...
43907,npk pelangi 16 16 16,pelangi 16 16 16,0.666667
43912,npk pelangi 13 6 27 4 fuse,npk pelangi 13 6 27 4,0.857143
43914,npk 16 16 16 rusia,npk pusri 16 16 16,0.500000
43926,sp 36 petrogress,sp 36 petro,0.500000


In [11]:
result = result_jaccard.merge(catalog[['Product SKU','cleaned_name']],how='left',right_on='cleaned_name',left_on='best_catalog_name')
result = result[['pos_cleaned_name','Product SKU']]
result = result.merge(pos[['Product Name','cleaned_name']],how='left',right_on='cleaned_name',left_on='pos_cleaned_name')
result = result[['Product Name','Product SKU']]
result = result.drop_duplicates()

In [12]:
result

,Product Name,Product SKU
0,nitrea,Urea Nitrea
1,Nitrea@25kg,Urea Nitrea
2,Nitrea@ 25kg,Urea Nitrea
3,Nitrea@50kg,Urea Nitrea
4,Nitrea@5kg,Urea Nitrea
...,...,...
4628,Urea prill Nitrea,Urea Nitrea
4629,NPK PELANGI 12-12-17-2 BLD NS @50KG,Pelangi 12-12-17-2
4635,NPK PELANGI 13-6-27-4 FUSE @50KG,NPK Pelangi 13-6-27-4
4636,npk 16-16-16 rusia,NPK Pusri 16-16-16


In [13]:
jaccard_5 = result_df[(result_df['jaccard_similarity_score'] < 0.5)].reset_index().drop('index',axis=1)

In [14]:
jaccard_5.to_excel('jaccard_5.xlsx',index=False)

**Stage II**

filtered product start with 'pupuk'

In [15]:
pos_pupuk = jaccard_5[jaccard_5['pos_cleaned_name'].str.startswith("pupuk")]

In [16]:
pos_pupuk['pos_cleaned_name'] = pos_pupuk['pos_cleaned_name'].str.split(n=1).str[-1]

C:\Users\Hanif\AppData\Local\Temp\ipykernel_286976\1519665598.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_pupuk['pos_cleaned_name'] = pos_pupuk['pos_cleaned_name'].str.split(n=1).str[-1]


In [17]:
# Create an empty list to store the results
result_data = []

for pos_pupuk_name in pos_pupuk['pos_cleaned_name']:
    best_similarity = 0
    best_catalog_name = ''

    for catalog_name in catalog['cleaned_name']:
        similarity = calculate_jaccard_similarity(pos_pupuk_name, catalog_name)
        if similarity > best_similarity:
            best_similarity = similarity
            best_catalog_name = catalog_name

    result_data.append([pos_pupuk_name, best_catalog_name, best_similarity])

# Create the result DataFrame from the result_data list
result_pupuk = pd.DataFrame(result_data, columns=['pos_cleaned_name', 'best_catalog_name', 'jaccard_similarity_score'])

In [18]:
result_pupuk = result_pupuk[result_pupuk['jaccard_similarity_score']>=0.5]

In [19]:
result_pupuk = result_pupuk.merge(catalog[['Product SKU','cleaned_name']],how='left',right_on='cleaned_name',left_on='best_catalog_name')
result_pupuk = result_pupuk[['pos_cleaned_name','Product SKU']]
result_pupuk = result_pupuk.merge(pos[['Product Name','cleaned_name']],how='left',right_on='cleaned_name',left_on='pos_cleaned_name')
result_pupuk = result_pupuk[['Product Name','Product SKU']]
result_pupuk = result_pupuk.drop_duplicates()

In [20]:
result = pd.concat([result_pupuk,result])

In [21]:
result

,Product Name,Product SKU
0,Cantik,CANTIK CALCINIT
1,ZK,ZK Petro
2,za,ZA Petro
3,ZA@ 50kg,ZA Petro
4,UREA PIM NON SUBSIDI,Urea PIM
...,...,...
4628,Urea prill Nitrea,Urea Nitrea
4629,NPK PELANGI 12-12-17-2 BLD NS @50KG,Pelangi 12-12-17-2
4635,NPK PELANGI 13-6-27-4 FUSE @50KG,NPK Pelangi 13-6-27-4
4636,npk 16-16-16 rusia,NPK Pusri 16-16-16


In [7]:
result.to_excel('result.xlsx',index=False)

NameError: name 'result' is not defined

In [23]:
# Create a boolean mask to filter rows in 'pos' where 'Product Name' is not in 'result'
mask = ~pos['Product Name'].isin(result['Product Name'])

# Apply the mask to filter the rows
filtered_pos = pos[mask]

filtered_pos

,Product Name,cleaned_name
0,Pupuk Urea N 46%,pupuk urea n 46%
1,Pupuk Amonium Sulfat ZA,pupuk amonium sulfat za
2,Pupuk Super Fosfat SP-36,pupuk super fosfat sp 36
3,Pupuk NPK Phonska,pupuk npk phonska
4,Pupuk NPK Formula Khusus,pupuk npk formula khusus
...,...,...
43997,Extra one 680 EC @ 250 ml,extra one 680 ec
43998,Extra One 680 SC @ 500 ml,extra one 680 sc
43999,JARING ARWANA @ 100 METER,jaring arwana
44000,Terong Puma F1 @ 5 gram,terong puma f1


**Stage III**

Get new product SKU with one word POS product

In [24]:
# Create a boolean mask to filter rows where 'Product Name' has a length of 1
mask = filtered_pos['Product Name'].str.match(r'^\S+$')
# Apply the mask to select the rows
selected_rows = filtered_pos[mask]

In [66]:
#selected_rows.to_excel('one_word.xlsx',index=False)

Processing with Excel

In [8]:
new_product_sku = pd.read_excel('new_product_sku.xlsx')
result = pd.read_excel('result.xlsx')

In [13]:
new_product_sku['New Product SKU'] = new_product_sku['New Product SKU'].str.title()

In [14]:
new_product_sku = new_product_sku.dropna()

In [15]:
new_product_sku = new_product_sku.rename(columns={'New Product SKU':'Product SKU'})
new_product_sku = new_product_sku[['Product Name','Product SKU']]
result = pd.concat([result,new_product_sku])

In [16]:
result = result.drop_duplicates()

In [17]:
new_catalog = pd.DataFrame({'Product SKU':result['Product SKU'].unique(),'Brand':'','Type':'','Formula':''})

In [18]:
new_catalog

,Product SKU,Brand,Type,Formula
0,CANTIK CALCINIT,,,
1,ZK Petro,,,
2,ZA Petro,,,
3,Urea PIM,,,
4,Mestac,,,
...,...,...,...,...
4460,Goodfer,,,
4461,Greenbig,,,
4462,Gramoxaon,,,
4463,Propel,,,


In [19]:
new_catalog['cleaned_name'] = new_catalog['Product SKU'].str.lower().str.replace('-', ' ').str.replace(',', ' ').str.replace(r'@.*', '', regex=True)
new_catalog['cleaned_name'] = new_catalog['cleaned_name'].astype(str)

In [20]:
new_catalog

,Product SKU,Brand,Type,Formula,cleaned_name
0,CANTIK CALCINIT,,,,cantik calcinit
1,ZK Petro,,,,zk petro
2,ZA Petro,,,,za petro
3,Urea PIM,,,,urea pim
4,Mestac,,,,mestac
...,...,...,...,...,...
4460,Goodfer,,,,goodfer
4461,Greenbig,,,,greenbig
4462,Gramoxaon,,,,gramoxaon
4463,Propel,,,,propel


In [66]:
catalog = pd.concat([catalog,new_catalog])

In [67]:
catalog

,Product SKU,Brand,Type,Formula,cleaned_name
0,Urea Petro,PIHC,Urea,NaN,urea petro
1,Urea PIM,PIHC,Urea,NaN,urea pim
2,Urea Nitrea,PIHC,Urea,NaN,urea nitrea
3,Urea Daun Buah,PIHC,Urea,NaN,urea daun buah
4,Urea Pusri,PIHC,Urea,NaN,urea pusri
...,...,...,...,...,...
4460,Goodfer,,,,goodfer
4461,Greenbig,,,,greenbig
4462,Gramoxaon,,,,gramoxaon
4463,Propel,,,,propel


## FuzzyWuzzy

In [69]:
# Create a boolean mask to filter rows in 'pos' where 'Product Name' is not in 'result'
mask = ~pos['Product Name'].isin(result['Product Name'])

# Apply the mask to filter the rows
filtered_pos = pos[mask]

filtered_pos

,Product Name,cleaned_name
0,Pupuk Urea N 46%,pupuk urea n 46%
1,Pupuk Amonium Sulfat ZA,pupuk amonium sulfat za
2,Pupuk Super Fosfat SP-36,pupuk super fosfat sp 36
3,Pupuk NPK Phonska,pupuk npk phonska
4,Pupuk NPK Formula Khusus,pupuk npk formula khusus
...,...,...
43997,Extra one 680 EC @ 250 ml,extra one 680 ec
43998,Extra One 680 SC @ 500 ml,extra one 680 sc
43999,JARING ARWANA @ 100 METER,jaring arwana
44000,Terong Puma F1 @ 5 gram,terong puma f1


In [76]:
from fuzzywuzzy import fuzz

C:\Users\Hanif\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [78]:
# Function to find the most similar SKU based on name
def find_similar_sku(product_name):
    max_similarity = 0
    best_match_sku = None

    for _, catalog_row in catalog.iterrows():
        similarity = fuzz.ratio(product_name, catalog_row['cleaned_name'])
        if similarity > max_similarity:
            max_similarity = similarity
            best_match_sku = catalog_row['cleaned_name']

    if max_similarity < threshold:
        return None  # You can set a threshold for similarity and propose a new SKU

    return best_match_sku

# Map product names to SKUs
threshold = 70  # Adjust as needed
filtered_pos['Product SKU'] = filtered_pos['cleaned_name'].apply(find_similar_sku)

C:\Users\Hanif\AppData\Local\Temp\ipykernel_286976\1884629666.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_pos['Product SKU'] = filtered_pos['cleaned_name'].apply(find_similar_sku)


In [103]:
result_fuzzy = filtered_pos[filtered_pos['Product SKU'].notnull()]

In [87]:
filtered_pos.to_excel('result_fuzzy.xlsx',index=False)

In [109]:
result_fuzzy = result_fuzzy.rename(columns={'Product SKU':'best_catalog_name','cleaned_name':'pos_cleaned_name'}).drop('Product Name',axis=1)

In [110]:
result_fuzzy

,pos_cleaned_name,best_catalog_name
1,pupuk amonium sulfat za,ammonium sulfate
10,abacel 250ml,abacell
20,starban 100ml,starban
51,sampurna b,sampurna
56,broadpls/ob jkt,broadpls
...,...,...
43983,sprayer cba,hndsprayercba
43984,sarnet,sarento
43985,promotop 20 l,promotop
43986,bablas 20 l,bablas


In [111]:
result_fuzzy = result_fuzzy.merge(catalog[['Product SKU','cleaned_name']],how='left',right_on='cleaned_name',left_on='best_catalog_name')
result_fuzzy = result_fuzzy[['pos_cleaned_name','Product SKU']]
result_fuzzy = result_fuzzy.merge(pos[['Product Name','cleaned_name']],how='left',right_on='cleaned_name',left_on='pos_cleaned_name')
result_fuzzy = result_fuzzy[['Product Name','Product SKU']]
result_fuzzy = result_fuzzy.drop_duplicates()

In [112]:
result_fuzzy

,Product Name,Product SKU
0,Pupuk Amonium Sulfat ZA,Ammonium Sulfate
1,abacel 250ml,Abacell
2,starban 100ml,Starban
3,sampurna B,Sampurna
4,Broadpls/ob jkt,Broadpls
...,...,...
18417,SPRAYER CBA,Hndsprayercba
18418,SARNET,Sarento
18419,PROMOTOP 20 L,Promotop
18420,BABLAS 20 L,Bablas


In [116]:
result = pd.concat([result,result_fuzzy]).drop_duplicates()

In [117]:
result

,Product Name,Product SKU
0,Cantik,CANTIK CALCINIT
1,ZK,ZK Petro
2,za,ZA Petro
3,ZA@ 50kg,ZA Petro
4,UREA PIM NON SUBSIDI,Urea PIM
...,...,...
18417,SPRAYER CBA,Hndsprayercba
18418,SARNET,Sarento
18419,PROMOTOP 20 L,Promotop
18420,BABLAS 20 L,Bablas


In [118]:
result.to_excel('result_fuzzy_jaccard.xlsx',index=False)

In [54]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Reset the index of the catalog DataFrame
catalog = catalog.reset_index(drop=True)

# Concatenate the cleaned names from both DataFrames
combined_cleaned_names = pd.concat([catalog['cleaned_name'], pos['cleaned_name']])

# Create a CountVectorizer to convert text into a bag-of-words representation
vectorizer = CountVectorizer().fit_transform(combined_cleaned_names)

# Calculate cosine similarities
cosine_similarities = cosine_similarity(vectorizer)

# Create an empty DataFrame to store the results
result_data = []

# Iterate through each pos['cleaned_name']
for i, pos_name in enumerate(pos['cleaned_name']):
    # Find the index of the most similar catalog name
    most_similar_index = cosine_similarities[i][:len(catalog)].argmax()
    
    # Get the corresponding catalog name and cosine similarity score
    best_catalog_name = catalog['cleaned_name'].iloc[most_similar_index]
    cosine_similarity_score = cosine_similarities[i][most_similar_index]
    
    result_data.append({'pos_cleaned_name': pos_name, 'best_catalog_name': best_catalog_name, 'cosine_similarity_score': cosine_similarity_score})

# Create the result DataFrame from the result_data list
result_df_cosine = pd.DataFrame(result_data, columns=['pos_cleaned_name', 'best_catalog_name', 'cosine_similarity_score'])

# The result DataFrame 'result_df_cosine' now contains the cosine similarity results

In [59]:
result_df_cosine[result_df_cosine['cosine_similarity_score']>=0.8]

,pos_cleaned_name,best_catalog_name,cosine_similarity_score
0,pupuk urea n 46%,urea petro,1.000000
1,pupuk amonium sulfat za,urea pim,1.000000
2,pupuk super fosfat sp 36,urea nitrea,1.000000
3,pupuk npk phonska,urea daun buah,1.000000
4,pupuk npk formula khusus,urea pusri,1.000000
...,...,...,...
43817,alika ( 250 ml ),npk pim 15 15 15,1.000000
43849,swan spray manual 14,mutiara grower 15 09 20+te,0.866025
43894,yara liva,npk kujang 30 6 8,0.866025
43912,npk pelangi 13 6 27 4 fuse,pelangi agro 20 10 10,0.857143
